In [12]:
!pip install --upgrade pip
!pip install uvicorn
!pip install fastapi
!pip install pyngrok

In [13]:
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rioxarray
!pip install rasterio
!pip install geopandas
!pip install geojson
!pip install shapely

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgdal-dev is already the newest version (3.4.3+dfsg-1~jammy0).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading p

In [14]:
!mkdir foss4g

In [26]:
ls

coverage.json  ndvi.tif


In [29]:
# put the parameter in your request
from fastapi import FastAPI, Query
import geojson
import shapely.wkt
import json
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
from rioxarray import open_rasterio
import ast
import os
import json
from osgeo import gdal

app = FastAPI()
@app.get('/area')
async def root(coords: str = Query(None, description="A test parameter")):

  wkt_data = shapely.wkt.loads(coords)
  wkt_data2 = geojson.Feature(geometry=wkt_data, properties={})

  dump_json = json.dumps(wkt_data2)
  gdf = gpd.read_file(dump_json)

  # Open the raster file to be clipped
  raster_file = 'ndvi.tif'
  src = rasterio.open(raster_file)
  print(src)

  # Clip the raster using the GeoJSON geometry
  clipped_data, clipped_transform = mask(src, gdf.geometry, crop=True)

  clipped_meta = src.meta
  clipped_meta.update({
      'height': clipped_data.shape[1],
      'width': clipped_data.shape[2],
      'transform': clipped_transform
  })

  # Define the output file name
  output_file = 'clipped_raster.tif'

  # Create a new raster file with the clipped data and updated metadata
  with rasterio.open(output_file, 'w', **clipped_meta) as dst:
      dst.write(clipped_data)

  print(f"Clipped raster saved to {output_file}")


  # Open the clipped raster file
  output_file = 'clipped_raster.tif'
  with rasterio.open(output_file) as ds_nir_org:
      # Read the pixel values
    data = ds_nir_org.read(1)

    # Specify the background value (replace 0 with your background value)
    background_value = 0.0

    # Mask the background values
    masked_data = np.ma.masked_equal(data, background_value)

    # Convert the masked data to a list
    text = masked_data.tolist()

    text2 = str(text).replace('[', '')
    text3 = str(text2).replace(']', '')
    text4 = str(text3).replace('Nan', str(None))


    arr = ast.literal_eval(text4)

    my_list = list(arr)


    dateImage = ['2010-01-01T00:12:20Z']
    crt = len(dateImage)

    # Opening JSON file
    f = open('coverage.json')

    # see detail into file coverage.json

    covJSON = json.load(f)

    covJSON['ranges']['data']['values'] = my_list

    get_file = output_file
    dataset = gdal.Open(get_file)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize


    def GetExtent(ds):
        xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
        width, height = ds.RasterXSize, ds.RasterYSize
        xmax = xmin + width * xpixel
        ymin = ymax + height * ypixel

        return [xmin, ymin, xmax, ymax]
    ds = dataset
    extent = GetExtent(ds)

    axes = {
        "x": {
            "start": extent[0],
            "stop": extent[2],
            "num": cols
        },
        "y": {
            "start": extent[3],
            "stop": extent[1],
            "num": rows
        },
        "t": {"values": dateImage}
    }

    covJSON['domain']['axes'] = axes

    shape = [crt, rows, cols]

    covJSON['ranges']['data']['shape'] = shape

    msgRetren = covJSON


    return msgRetren

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://43be-35-245-5-82.ngrok.io


INFO:     Started server process [201]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


<open DatasetReader name='ndvi.tif' mode='r'>
Clipped raster saved to clipped_raster.tif
INFO:     175.192.226.112:0 - "GET /area?coords=POLYGON%20((100.40375586163655%2014.853583642788516,%20100.40492627187803%2014.851100089349307,%20100.40795221055109%2014.852584512094582,%20100.40655342757957%2014.855039518942535,%20100.40375586163655%2014.853583642788516)) HTTP/1.1" 200 OK
INFO:     175.192.226.112:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


url for test ==> {Public URL}/_area?coords=POLYGON%20((100.40375586163655%2014.853583642788516,%20100.40492627187803%2014.851100089349307,%20100.40795221055109%2014.852584512094582,%20100.40655342757957%2014.855039518942535,%20100.40375586163655%2014.853583642788516))

area wkt for test ==>

wkt_1 = "POLYGON ((100.40375586163655 14.853583642788516, 100.40492627187803 14.851100089349307, 100.40795221055109 14.852584512094582, 100.40655342757957 14.855039518942535, 100.40375586163655 14.853583642788516))"

wkt_2 = "POLYGON ((100.41271949129072 14.85555335758513, 100.4110066958154 14.85398329506609, 100.41357588902837 14.851271368896839, 100.41506031177364 14.852670151868347, 100.41271949129072 14.85555335758513))"